The following is a collection of text written during analysis, assembled here for ease of reading and lightly edited. 

## Data Review

The data looks pretty well filled out, as the counts across all fields match within each month. 
 - **Vendor ID** - The mean is really close across months, meaning we have similar splits from the two systems
 - The months seem to have similar numbers of records.
 - **Passenger count** has extremely similar mean and variance.
 - **Trip distance** in June has a much higher variance than the other two months, but similar means. Perhaps something to look out for, but this is one of the major variables we want to predict on, and distance is a factor in the calculation of fare, which influences tip. So I am not too concerned about this one. Should check later for correlation between distance and fare amount. They are probably confounded. The maxima are around 9,000 miles, which is clearly too large.
 - **Rate code** is interesting. The mean in all months is very close to 1, indicating the vast majority of fares are standard fares, but the variance changes significantly in June. Perhaps this is more people going to and from the airport in a summer month. I was thinking of creating binary fields for airport, maybe I should add more for some of these other types
 - **Pick Up Location** and **Drop Off Location** seem pretty similar. These are zones and the closest we have to a spatial component. Looks like there are up to 265 different zones throughout the city. This might be a good factor to indicate tip amount, as it could be a proxy for some sort of demographic. But it would need to be converted into something other than a continuous variable if we do a regression. I don't want to create 264 different binary variables. It would be nicei f there were a structure to these and I could link to something like census tract or neighborhood, but given time constraints and the wealkth of other information that are probably stronger signifiers for tip amount (fare amount, distance, fare type), I probably won't get into this.
 - **Payment type** does not vary across month. This will need to be converted into binary variables for a regression.
 - **Fare amount** shows similar means but very different standard deviations. It always has a mean of about 13, but the standard deviation swings from 55 in March to 200 in June to 130 in November. This is probably due to some extremely high fares. I should look at the distributions of these. Sure enough, the 25th, 50th, and 75th percentile breaks are extremely similar but the maxima vary.
 - **Extra** looks pretty similar but drops in November.
 - **MTA Tax** has a similar mean across months but the variance changes. However, this is a pretty small amount (tenths of a cent). Probably not a big deal, and this is probably not something we'll use to predict tip anyway since it's a standard addition.
 - **Tip Amount** - This is consistent across months. That's good since this is what we are trying to predict. Also good that the distribution doesn't seem to be affected by the total's large numbers. Perhaps those are errors.
 - **Tolls Amount** - Also pretty consistent. Slight drop in March in the mean, but variance looks similar.
 - **Total Amount** - seems to vary much like fare amount, which makes sense if it is the sum of all the monetary fields.
 
So overall I am not concerned with seasonality across the different months and feel good about using the data simply merged together. Month has no discernable effect on tip here. 

Also, the values and distributions of the fields seem to make sense. There are some large outliers on the fare amount, which are on the order of $100,000. If these were valid data points, they must be very extreme cases. Also, there are a lot of neative values, which seems strange and indicative of something wrong in the data.

## Exploratory Data Analysis

Ultimately, I will probably do some kind of linear regression, as the tip is a continuous variable and I want to predict it as such. I could alternatively try to bin it and predict a round number as a categorical variable (1.00, 1.50, 2.00, etc), but I'll probably try as a continuous variable first. Either way, there are several variables that are presented as numbers that will need to be converted to binary.

Looking at the distributions and comparing to the data dictionary, we see there are some error values that are affecting our distributions. We should probably filter some of these out. Pickup and dropoff locations look good. 
- Rate Code ID should have a max of 6
- Extras should be 50 cents or a dollar, so a max of 1 and a min of 0 there works
- MTA tax should only be 50 cents. we're not going to use that value anyway, since it should apply to all trips. But the scale shows some strange values. Below we see some high fares in those cases. So probably good to filter out anyway.
- Passenger count has a single record with a value over 10. can remove that one.
- Distributions of toll amounts, tip amount, fare amount, and trip distance would be good to look at in more detail, but none should be negative

I think we can safely remove any trip with a distance over 400 miles, though even that is probably excessive. 400 miles would get someone well past DC or Boston. The tip amount seems too high but I don't have a great reason to cut any of these out yet. Perhaps other filters will help. I think fares over 500 can probably be removed as well.

The amounts still have some pretty long tails, but this generally looks better. The tails are probably erroneous, but without a better justification for removing points, I would rather keep the long tails in. Also, it seems like a potential outcome that we should be able to predict. The model will be less reliable at these extremes, but I'd rather at this point include and have some weird output rather than no output. There is enough data where we expect to see reasonable trips that those trips shouldn't be affected.

### Fare Types
So when looking withiin fare types, we do see some differences. There are far more of fare type 1 than any other record. That makes sense. Type 2 and 3 originate at JFK and Newark airport. There are only a couple fares originating in Newark, which is strange. Types 4 and 5 seem to follow similar distributions, and they aren't that far off from 1, but those are typically bigger fares, as they either go outside the city (4 is Nassau/Westchester and 5 is a negotiated fare), so we would expect that. Type 6 is a group ride, which is kind of strange to see so many small tips, but perhaps there are just no long rides. I'll look at these with respect to fare amount as well.

Fare type 2 (JFK) is a set rate, so the distribution of tips is not dependent on the amount. This is important for prediction. 4 and 5 seem to have steeper lines than type 1. type 6 is for group rates, which just seems like there is invalid data. Everything is either 0 tip or 0 fare (or both). Also, we see concentrations in vertical lines at round numbers. I'd be curious to see if that is more common with cash tips or card payments

### Payment Method
Most of the valid values seem to be type 1, or credit card. Cash (type 2) has a lot of trips but hardly any non-zero tips. My guess is that cash tips are being paid, but not being reported. We probably should not include these data points in our model if we want to get an accurate reflection of what someone should tip, or what amount of tip the driver should receive. If we want to predict what is being reported, these would be valuable points, but that is not our goal.

Again, we see cash payments having non-zero fares, but no tips are being reported. Types 3 (no charge) and 4 (dispute) show some tips and a somewhat linear relationship, but again, there are far more zero tips than non-zero. Perhaps we should filter out these trips for prediction as well, assuming there will be a charge and it will not be disputed.

### Temporal Review
So overall I am not concerned with seasonality across the different months and feel good about using the data simply merged together. Month has no discernable effect on tip here.

Plots for day of the week looked exactly the same. Nice idea, but not significant enough to put into the regression. I could check the pickup hour as well, but based on these results, I am not going to look at it because of time constraints.

Hourly records don't show any significant differences either. Certainly there is a difference in volume, but the relationship between tip and fare does not seem to be changing based on the hour of day.

### Correlation and Confounded Variables

As anticipated, trip distance, fare amount, tip amount, and total amount are highly correlated.

We have already looked at rate code, so I don't want to get into that. Besides, it's not a true continuous variable, so the correlation is meaningless. The negative correlation between tip amount and payment type is similar, and was discussed above because of the prevalence of tips only coming on credit cards.

There's an interesting correlation between passenger count and vendor id, but neiher show correlation with tip amount, so we will ignore. Possibly one system does not account for passenger count.

We will also ignore pickup and dropoff locations, as those are showing very little correlation to anything. And they are really categorical variables.

So that leaves us with distance, fare amount, and total amount to use for prediction. Because these three are highly correlated with each other, we should pick just one to use. I propose just using fare amount, as that is probably the best. However, it might be worth creating a field of all non-tip amounts and checking the correlations of that and fare amount against tip amount.

So the fare amount and the nontip total correlate very strongly, and the tip amount correlates slightly better with the nontip total than with just the fare amount. So it is probably worth using the nontip total in the regression. This makes sense to me, as tolls might end up being $20, and a lot of people probably tip on the total number as opposed to a subtotal.

### Findings
- We need to filter out lots of erroneous data.
- Credit card payments are the only reliable payment type for tip prediction. We will filter out all other types.
- Fare type as defined by atypical or out of zone pickup/dropoff locations are important to prediction as they show their own patterns.
- There is no pattern to tipping tied to day of the week, month of the year, or hour of the day in our dataset.

## Model Building

The tip on a taxi ride is a continuous variable, so I am going to try to predict it as a continuous variable, at least at first. Creating buckets could be helpful, especially because people often tip in round numbers anyway. But for now, I am going to use a linear regression model with the several factors I isolated through EDA. 

The **tip_amount** will be our label that we are trying to predict. The **nontip_total** represents a sum of all the costs associated with the ride not including the tip. This includes the fare and any tolls, taxes, or special fees. **JFK** indicates a ride going to or originating at JFK International Airport. These rides have set prices, and that will impact the tip. **long_dist** is also an indicator variable signifying rides that left the city. This is typically Nassau and Westchester counties. **negotiated** is a third indicator variable that denotes whether the fare was previously negotiated. For these three, a value of 1 is true.

### Results

A simple linear regression is not going to get much better than what I have already shown. Different random seeds produced R squared values between 0.55 and 0.60. This is probably to be expected as there is a lot of variance throughout the dataset, and the correlation coefficient of tip_amount and nontip_total was shown to be 0.59. Cross-validation could help stabilize this number, but given the size of the dataset (19M records for prediction), I am not concerned with cross-validation at the moment.

What we see in the coefficients of the linear model is that a tip is typically 17% of the pre-tip total cost. If the rider is coming from JFK, they will add on average 35 cents to their tip. For someone traveling to destinations outside the city, they will deduct 87 cents from that 17% number. This makes sense since a long distance ride probably has a higher fare and people are likely to tip at a lower percentage. Similarly, pre-negotiated rates are tipping $2.80 less than a trip of similar cost.

I also tried a stochastic gradient descent regressor and a decision tree regressor to see if I can improve the performance, but really, it would be good if there were ways to supplement the data and add features that might explain this variance.

Of the data we have, the pickup and dropoff locations might be helpful, so I tried adding them quickly to a tree regressor model. With some work of knowing where the zones are, we could probably use these to better predict tip amounts. Right now, they are numeric and being treated as such, which is inappropriate.

The two versions of the SGD regressor produced essentially the same results, even with more data. The R squared value is staying roughly the same. In the model where I gave it more data, the model performance is actually getting worse if you were to evaluate an adjusted R squared.

Also, by limiting the rides that I used to build the model based on the filters I set up, there are many conditions I am not testing for. So really, I should be bringing all the original data back in for testing, but I am currently trying to build something reliable off the good data that can be tested against (the cash data effectively has no labels to test). I know the accuracy will decrease if and when I bring in new data streams in a production environment, but for the purposes of time, I am working with the 19M records I imported here.

## Conclusion

Given the time limit on the problem, I was not able to find a model that would work better than a simple linear regression based primarily on the pre-tip trip cost. This regression model does a good job of predicting what a reasonable tip would be, but it does not do a very good job of predicting tipping behavior. So if our goal is to provide reasonable tip recommendations to serve as a guideline for riders, the model could do well. But if our goal is to successfully predict how each individual will tip, it will not generally succeed. This is largely because of non-uniform individual behavior (or possibly due to other factors we can't see in the data).

Ultimately, tipping varies from person to person, so that probably explains a lot of the variance. Having data on riders the way Uber or Lyft do might allow me to create tipping profiles of individual users to better predict a tip on each ride. Similarly, Uber and Lyft data on rider and driver ratings might also go a long way towards predicting the quality of the ride and therefore the tip. Once the company is up and running and generating data, I would want to try to improve the tipping recommendation model based on these parameters.